In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from imgaug import augmenters as iaa
import imgaug as ia
import gc
import csv
import keras
import pandas as pd
from keras.models import Sequential
import csv
from keras.layers import GlobalMaxPooling1D, Dense, Conv1D,Conv2D
from keras import regularizers,optimizers
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.models import Model
from keras.layers import (
    Input,
    Dense,
    Flatten,
    merge,
    Lambda
)
from __future__ import print_function

import numpy as np
import warnings
from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs
from keras.layers.convolutional import (
    Convolution2D,
    MaxPooling2D,
    AveragePooling2D,
    ZeroPadding2D
)
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
import keras.backend as K

from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np

from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalAveragePooling1D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing import image

# Evidently this model breaks Python's default recursion limit
# This is a theano issue
import sys
sys.setrecursionlimit(10000)

from keras.models import Model

Using TensorFlow backend.


In [ ]:
!cp '/content/drive/My Drive/MRNet.zip' '/content/MRNet.zip'

In [ ]:
!wget http://download.cs.stanford.edu/deep/MRNet-v1.0.zip
!unzip MRNet.zip
data_dir = 'content/MRNet-v1.0'

Archive:  MRNet.zip
warning [MRNet.zip]:  4294967296 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  4294967296
  (attempting to re-compensate)
   creating: MRNet-v1.0/
  inflating: MRNet-v1.0/valid-abnormal.csv  
   creating: MRNet-v1.0/valid/
   creating: MRNet-v1.0/valid/axial/
  inflating: MRNet-v1.0/valid/axial/1139.npy  
  inflating: MRNet-v1.0/valid/axial/1138.npy  
  inflating: MRNet-v1.0/valid/axial/1249.npy  
  inflating: MRNet-v1.0/valid/axial/1248.npy  
  inflating: MRNet-v1.0/valid/axial/1207.npy  
  inflating: MRNet-v1.0/valid/axial/1213.npy  
  inflating: MRNet-v1.0/valid/axial/1171.npy  
  inflating: MRNet-v1.0/valid/axial/1165.npy  
  inflating: MRNet-v1.0/valid/axial/1159.npy  
  inflating: MRNet-v1.0/valid/axial/1158.npy  
  inflating: MRNet-v1.0/valid/axial/1164.npy  
  inflating: MRNet-v1.0/valid/axial/1170.npy  
  inflating: MRNet-v1.0/valid/axial/1212.npy  
  inflating: MRNet-v1.0/valid

In [ ]:
def augment(image):
  aug=[]
  for s in range(len(image)):
      aug.append(ia.imresize_single_image(image[s], (227, 227)))
  aug=np.array(aug)
  return aug

def clear():
  aug= None
  gc.collect()

def add_rgb(grey_img):
  rgb_img = np.repeat(grey_img[..., np.newaxis], 3, -1)
  return rgb_img

def load_data(path):
  data = []
  for d in sorted(os.listdir(path)):
    if d!='.DS_Store':
      data.append(d)
      print("in iterator ", d)
      clear()
  return data
  
def load_labels(path):
  labels = []
  read = pd.read_csv(path, names=['num', 'hot'])
  labels = list(read['hot'])
  return labels


# AlexNet Feature Extractor Model

In [ ]:
#MRNet - Model
ALEXNet = Sequential()

# 1st Convolutional Layer
ALEXNet.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
ALEXNet.add(Activation('relu'))

# Max Pooling
ALEXNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
ALEXNet.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid'))
ALEXNet.add(Activation('relu'))

# Max Pooling
ALEXNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
ALEXNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
ALEXNet.add(Activation('relu'))

# 4th Convolutional Layer
ALEXNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
ALEXNet.add(Activation('relu'))

# 5th Convolutional Layer
ALEXNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
ALEXNet.add(Activation('relu'))

average_pool = Sequential()
average_pool.add(layers.AveragePooling2D())
average_pool.add(layers.Flatten())
#average_pool.add(layers.Dense(1, activation='sigmoid'))


MRNet = Sequential([
    ALEXNet, 
    average_pool])


MRNet.add(Dense(256, activation ='relu',kernel_constraint=keras.constraints.MaxNorm(max_value=2, axis=0)))
MRNet.add(Dense(1, activation ='sigmoid'))

sgd = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

ALEXNet.summary()
average_pool.summary()
MRNet.summary()

MRNet.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
activation_2 (Activation)    (None, 23, 23, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 384)        

In [ ]:
#Load Data
train_axial_dir='MRNet-v1.0/train/axial'
train_coronal_dir= 'MRNet-v1.0/train/coronal'
train_sagittal_dir= 'MRNet-v1.0/train/sagittal'

train_abnormal_labels= '/content/MRNet-v1.0/train-abnormal.csv'
train_acl_labels= '/content/MRNet-v1.0/train-acl.csv'
train_meniscus_labels= '/content/MRNet-v1.0/train-meniscus.csv'

In [ ]:
print("**************     load axial   **************** ")
train_axial_data = os.listdir(train_axial_dir)

print("**************     load cronal   **************** ")
train_coronal_data = os.listdir(train_coronal_dir)

print("**************     load sagital   **************** ")
train_sagittal_data = os.listdir(train_sagittal_dir)

print("**************  load abnormal labels   **************** ")
train_abnormal_labels = load_labels(train_abnormal_labels)

print("**************  load ACL labels   **************** ")
train_acl_labels = load_labels(train_acl_labels)

print("**************  load Meniscus labels   **************** ")
train_meniscus_labels = load_labels(train_meniscus_labels)

**************     load axial   **************** 
**************     load cronal   **************** 
**************     load sagital   **************** 
**************  load abnormal labels   **************** 
**************  load ACL labels   **************** 
**************  load Meniscus labels   **************** 


In [ ]:

val_axial_dir = 'MRNet-v1.0/valid/axial'
val_coronal_dir = 'MRNet-v1.0/valid/coronal'
val_sagittal_dir = 'MRNet-v1.0/valid/sagittal'

val_abnormal_labels= '/content/MRNet-v1.0/valid-abnormal.csv'
val_acl_labels= '/content/MRNet-v1.0/valid-acl.csv'
val_meniscus_labels= '/content/MRNet-v1.0/valid-meniscus.csv'

val_axial_data = os.listdir(val_axial_dir)
val_coronal_data = os.listdir(val_coronal_dir)
val_sagittal_data =  os.listdir(val_sagittal_dir)


val_abnormal_labels= load_labels(val_abnormal_labels)
val_meniscus_labels= load_labels(val_meniscus_labels)
val_acl_labels= load_labels(val_acl_labels)


In [ ]:
!mv '/content/MRNet-v1.0.zip' 'drive/My Drive/MRNet.zip'

In [ ]:
#Hyperparameters 
def Average(lst): 
    return sum(lst) / len(lst) 
  
epoch=50
batch_size = 48

epochs=[0]*epoch
for i in range(0,epoch):
  epochs[i]=i

In [ ]:
import random as random

axialt_length = len(train_axial_data)/batch_size + len(train_axial_data)%batch_size
coronalt_length = len(train_coronal_data)/batch_size + len(train_coronal_data)%batch_size
saggitalt_length= len(train_sagittal_data)/batch_size + len(train_sagittal_data)%batch_size


axialv_length = len(val_axial_data)/batch_size + len(val_axial_data)%batch_size
coronalv_length = len(val_coronal_data)/batch_size + len(val_coronal_data)%batch_size
saggitalv_length = len(val_sagittal_data)/batch_size + len(val_sagittal_data)%batch_size


def data_gen(data,label,path,data_length,batch_size=48):
  all_data = list(zip(data,label))
  random.shuffle(all_data)

  imgs = np.zeros(shape=(batch_size,227,227,3))
  label= np.zeros(shape=(batch_size,1))

  j=0

  while(True):
    chunk = all_data[j*batch_size:(j+1)*batch_size]  
    for i,pair in enumerate(chunk):
      if(pair[0] != '.DS_Store'):
        img = np.load(os.path.join(path,pair[0]),allow_pickle=True)
        img = img.astype(np.uint8)
        img = augment(img)
        img = add_rgb(img)
        imgs[i] = img[0,:,:,:]
        label[i] = pair[1]

        #yield (img_aug[0],pair[1])#np.repeat(pair[1],img_aug.shape[0]))
    yield (imgs,label)  
    j += 1
    

    
axial_abnormal = data_gen(train_axial_data , train_abnormal_labels,train_axial_dir,axialt_length)
coronal_abnormal = data_gen(train_coronal_data,train_abnormal_labels,train_coronal_dir,coronalt_length)
sagittal_abnormal= data_gen(train_sagittal_data,train_abnormal_labels,train_sagittal_dir,saggitalt_length)

axial_abnormal_val = data_gen(val_axial_data , val_abnormal_labels,val_axial_dir,axialv_length)
coronal_abnormal_val   = data_gen(val_coronal_data , val_abnormal_labels,val_coronal_dir,coronalv_length)
sagittal_abnormal_val  = data_gen(val_sagittal_data , val_abnormal_labels,val_sagittal_dir,saggitalv_length)

##########################################################
##########################################################

axial_acl = data_gen(train_axial_data , train_acl_labels,train_axial_dir,axialt_length)
coronal_acl = data_gen(train_coronal_data,train_acl_labels,train_coronal_dir,coronalt_length)
sagittal_acl = data_gen(train_sagittal_data,train_acl_labels,train_sagittal_dir,saggitalt_length)

axial_acl_val = data_gen(val_axial_data , val_acl_labels,val_axial_dir,axialv_length)
coronal_acl_val = data_gen(val_coronal_data , val_acl_labels,val_axial_dir,coronalv_length)
sagittal_acl_val = data_gen(val_sagittal_data , val_acl_labels,val_axial_dir,saggitalv_length)

##########################################################
##########################################################

axial_meniscus = data_gen(train_axial_data , train_meniscus_labels,train_axial_dir,axialt_length)
coronal_meniscus = data_gen(train_coronal_data,train_meniscus_labels,train_coronal_dir,coronalt_length)
sagittal_meniscus = data_gen(train_sagittal_data,train_meniscus_labels,train_sagittal_dir,saggitalt_length)

axial_meniscus_val = data_gen(val_axial_data , val_meniscus_labels,val_axial_dir,axialv_length)
coronal_meniscus_val = data_gen(val_coronal_data , val_meniscus_labels,val_axial_dir,coronalv_length)
sagittal_meniscus_val = data_gen(val_sagittal_data , val_meniscus_labels,val_axial_dir,saggitalv_length)


# Abnormal Section

In [ ]:
abnormalAccuracys=[]
axial_abnormal_output = MRNet.fit_generator(axial_abnormal, epochs =epoch,steps_per_epoch=axialt_length,max_queue_size=axialt_length)
axial_abnormal_valacc = MRNet.evaluate_generator(axial_abnormal_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)

abnormalAccuracys.append(axial_abnormal_valacc[1])
#plot_graphs(axial_abnormal_valacc)


Epoch 1/5
1131/1131 [==============================] - 199s 176ms/step - loss: 0.4972 - accuracy: 0.8132
Epoch 2/5
1131/1131 [==============================] - 193s 171ms/step - loss: 0.4820 - accuracy: 0.8132
Epoch 3/5
1131/1131 [==============================] - 193s 171ms/step - loss: 0.4707 - accuracy: 0.8133
Epoch 4/5
1131/1131 [==============================] - 197s 174ms/step - loss: 0.4601 - accuracy: 0.8142
Epoch 5/5
120/120 [==============================] - 15s 125ms/step


In [ ]:
coronal_abnormal_output = MRNet.fit_generator(coronal_abnormal, epochs =epoch,steps_per_epoch= coronalt_length)
coronal_abnormal_valacc = MRNet.evaluate_generator(coronal_abnormal_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
abnormalAccuracys.append(coronal_abnormal_valacc[1])
#plot_graphs(coronal_abnormal_valacc)

Epoch 1/5
1131/1131 [==============================] - 174s 154ms/step - loss: 0.4970 - accuracy: 0.8164
Epoch 2/5
1131/1131 [==============================] - 168s 149ms/step - loss: 0.4852 - accuracy: 0.8151
Epoch 3/5
1131/1131 [==============================] - 168s 148ms/step - loss: 0.4756 - accuracy: 0.8164
Epoch 4/5
1131/1131 [==============================] - 174s 154ms/step - loss: 0.4970 - accuracy: 0.8164
Epoch 2/5
1131/1131 [==============================] - 168s 149ms/step - loss: 0.4852 - accuracy: 0.8151
Epoch 3/5
1131/1131 [==============================] - 168s 148ms/step - loss: 0.4756 - accuracy: 0.8164
Epoch 4/5
1131/1131 [==============================] - 167s 148ms/step - loss: 0.4675 - accuracy: 0.8164
Epoch 5/5
Epoch 5/5
120/120 [==============================] - 15s 125ms/step


In [ ]:
sagittal_abnormal_out = MRNet.fit_generator(sagittal_abnormal, epochs =epoch,steps_per_epoch= coronalt_length)
sagittal_abnormal_valacc= MRNet.evaluate(sagittal_abnormal_val,steps=saggitalv_length ,verbose=1,max_queue_size=50,use_multiprocessing=False)
abnormalAccuracys.append(sagittal_abnormal_valacc[1])
#plot_graphs(sagittal_abnormal_valacc)

Epoch 1/5
1131/1131 [==============================] - 194s 171ms/step - loss: 0.4971 - accuracy: 0.8120
Epoch 2/5
Epoch 2/5
1131/1131 [==============================] - 183s 162ms/step - loss: 0.4820 - accuracy: 0.8124
Epoch 3/5
Epoch 3/5
1131/1131 [==============================] - 184s 163ms/step - loss: 0.4735 - accuracy: 0.8111
Epoch 4/5
1131/1131 [==============================] - 185s 164ms/step - loss: 0.4620 - accuracy: 0.8126
Epoch 5/5
Epoch 5/5
120/120 [==============================] - 15s 129ms/step


# ACL Section

In [ ]:
aclAccuracys=[]
axial_acl_output = MRNet.fit_generator(axial_acl, epochs =epoch,steps_per_epoch= axialt_length)
axial_acl_valacc = MRNet.evaluate_generator(axial_acl_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(axial_acl_valacc[1]) 

1131/1131 [==============================] - 207s 183ms/step - loss: 0.5337 - accuracy: 0.7922
Epoch 2/5
Epoch 2/5
1131/1131 [==============================] - 203s 179ms/step - loss: 0.4884 - accuracy: 0.8126
Epoch 3/5
Epoch 3/5
1131/1131 [==============================] - 201s 178ms/step - loss: 0.4764 - accuracy: 0.8131
Epoch 4/5
Epoch 4/5
1131/1131 [==============================] - 202s 179ms/step - loss: 0.4661 - accuracy: 0.8138
Epoch 5/5
Epoch 5/5
120/120 [==============================] - 17s 144ms/step


In [ ]:
coronal_acl_out = MRNet.fit_generator(coronal_acl, epochs =epoch,steps_per_epoch= coronalt_length)
coronal_acl_valacc = MRNet.evaluate_generator(coronal_acl_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(coronal_acl_valacc[1])
#plot_graphs(coronal_acl)

1131/1131 [==============================] - 184s 163ms/step - loss: 0.5078 - accuracy: 0.8084
Epoch 2/5
Epoch 2/5
1131/1131 [==============================] - 177s 157ms/step - loss: 0.4875 - accuracy: 0.8089
Epoch 3/5
Epoch 3/5
1131/1131 [==============================] - 178s 158ms/step - loss: 0.4774 - accuracy: 0.8084
Epoch 4/5
Epoch 4/5
1131/1131 [==============================] - 181s 160ms/step - loss: 0.4660 - accuracy: 0.8095
Epoch 5/5
Epoch 5/5
120/120 [==============================] - 16s 136ms/step


In [ ]:
sagittal_acl_out = MRNet.fit_generator(sagittal_acl, epochs =epoch,steps_per_epoch= saggitalt_length)
sagittal_acl_valacc = MRNet.evaluate_generator(sagittal_acl_val,steps=saggitalv_length ,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(sagittal_acl_valacc[1])
#plot_graphs(sagittal_acl)

Epoch 1/5
1131/1131 [==============================] - 190s 168ms/step - loss: 0.5067 - accuracy: 0.8107
Epoch 2/5
Epoch 2/5
1131/1131 [==============================] - 186s 165ms/step - loss: 0.4840 - accuracy: 0.8110
Epoch 3/5
Epoch 3/5
1131/1131 [==============================] - 190s 168ms/step - loss: 0.4714 - accuracy: 0.8115
Epoch 4/5
1131/1131 [==============================] - 191s 169ms/step - loss: 0.4588 - accuracy: 0.8125
Epoch 5/5
Epoch 5/5
120/120 [==============================] - 17s 141ms/step


# Meniscus Section

In [ ]:
meniscusAccuracys=[]
axial_meniscus_out = MRNet.fit_generator(axial_meniscus, epochs =epoch,steps_per_epoch= axialt_length)
axial_meniscus_valacc = MRNet.evaluate_generator(axial_meniscus_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(axial_meniscus_valacc[1])
#plot_graphs(axial_meniscus)

Epoch 1/5
Epoch 1/5
1131/1131 [==============================] - 216s 191ms/step - loss: 0.6575 - accuracy: 0.6414
Epoch 2/5
1131/1131 [==============================] - 210s 186ms/step - loss: 0.6360 - accuracy: 0.6521
Epoch 3/5
Epoch 3/5
1131/1131 [==============================] - 206s 182ms/step - loss: 0.6234 - accuracy: 0.6585
Epoch 4/5
Epoch 4/5
1131/1131 [==============================] - 204s 180ms/step - loss: 0.6089 - accuracy: 0.6688
Epoch 5/5
Epoch 5/5
120/120 [==============================] - 16s 131ms/step


In [ ]:
coronal_meniscus_out = MRNet.fit_generator(coronal_meniscus, epochs =epoch,steps_per_epoch= axialt_length)
coronal_meniscus_valacc = MRNet.evaluate_generator(coronal_meniscus_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(coronal_meniscus_valacc[1])
#plot_graphs(coronal_meniscus)

1131/1131 [==============================] - 184s 163ms/step - loss: 0.6600 - accuracy: 0.6366
Epoch 2/5
1131/1131 [==============================] - 180s 159ms/step - loss: 0.6397 - accuracy: 0.6490
Epoch 3/5
Epoch 3/5
1131/1131 [==============================] - 181s 160ms/step - loss: 0.6268 - accuracy: 0.6573
Epoch 4/5
Epoch 4/5
1131/1131 [==============================] - 180s 159ms/step - loss: 0.6125 - accuracy: 0.6663
Epoch 5/5
Epoch 5/5
120/120 [==============================] - 16s 135ms/step


In [ ]:
sagittal_meniscus_out = MRNet.fit_generator(sagittal_meniscus, epochs =epoch,steps_per_epoch= axialt_length)
sagittal_meniscus_valacc = MRNet.evaluate_generator(sagittal_meniscus_val,steps=saggitalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(sagittal_meniscus_valacc[1])

Epoch 1/5
1131/1131 [==============================] - 192s 170ms/step - loss: 0.6538 - accuracy: 0.6362
Epoch 2/5
1131/1131 [==============================] - 188s 166ms/step - loss: 0.6319 - accuracy: 0.6531
Epoch 3/5
1131/1131 [==============================] - 184s 162ms/step - loss: 0.6151 - accuracy: 0.6622
Epoch 4/5
1131/1131 [==============================] - 184s 162ms/step - loss: 0.5958 - accuracy: 0.6781
Epoch 5/5
120/120 [==============================] - 17s 141ms/step


In [ ]:
#Final Model Predictions
def combine_accuracies(acc_list):
  weight = float(1/len(acc_list))

  weight_acc = 0
  for acc in acc_list:
    weight_acc += weight * acc
  
  return weight_acc

abnormal_acc = combine_accuracies(abnormalAccuracys)
acl_acc = combine_accuracies(aclAccuracys)
meniscus_acc = combine_accuracies(meniscusAccuracys)

In [ ]:
print('Abnormal Disease Dev-Set Accuracy : ',abnormal_acc * 100.0 ,'%')
print('ACL Disease Dev-Set Accuracy : ',acl_acc *100.0 ,'%')
print('Meniscus Disease Dev-Set Accuracy : ',meniscus_acc *100.0 ,'%')

ACL Disease Dev-Set Accuracy :  56.19232654571533 %
Meniscus Disease Dev-Set Accuracy :  55.1238477230072 %


# ResNet - Transfer Learning

In [ ]:
Resnet = keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=None, pooling='avg')
for layer in Resnet.layers:
  layer.trainable= False

In [ ]:
Extra = Sequential()
Extra.add(Dense(256, activation ='relu',kernel_constraint=keras.constraints.MaxNorm(max_value=2, axis=0)))
Extra.add(Dense(256, activation ='relu',kernel_constraint=keras.constraints.MaxNorm(max_value=2, axis=0)))
Extra.add(Dense(256, activation ='relu',kernel_constraint=keras.constraints.MaxNorm(max_value=2, axis=0)))
Extra.add(Dense(1, activation ='sigmoid'))

MRRes_Abnormal_axial = Sequential([
    Resnet,
    Extra ])

MRRes_Abnormal_coronal = Sequential([
    Resnet,
    Extra ])

MRRes_Abnormal_sagittal = Sequential([
    Resnet,
    Extra ])


MRRes_Acl_axial = Sequential([
    Resnet,
    Extra ])


MRRes_Acl_coronal = Sequential([
    Resnet,
    Extra ])


MRRes_Acl_sagittal = Sequential([
    Resnet,
    Extra ])


MRRes_Meniscus_axial = Sequential([
    Resnet,
    Extra ])

MRRes_Meniscus_coronal = Sequential([
    Resnet,
    Extra ])

MRRes_Meniscus_sagittal = Sequential([
    Resnet,
    Extra ])

sgd = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

MRRes_Abnormal_axial.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Abnormal_coronal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Abnormal_sagittal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])


MRRes_Acl_axial.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Acl_coronal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Acl_sagittal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])

MRRes_Meniscus_axial.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Meniscus_coronal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Meniscus_sagittal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
#abnormalAccuracys=[]
#axial_abnormal_output = MRRes_Abnormal_axial.fit_generator(axial_abnormal, epochs =50,steps_per_epoch=axialt_length,max_queue_size=axialt_length)
#axial_abnormal_valacc = MRRes_Abnormal_axial.evaluate_generator(axial_abnormal_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)

#abnormalAccuracys.append(axial_abnormal_valacc[1])

#coronal_abnormal_output = MRRes_Abnormal_coronal.fit_generator(coronal_abnormal, epochs =50,steps_per_epoch= coronalt_length)
#coronal_abnormal_valacc = MRRes_Abnormal_coronal.evaluate_generator(coronal_abnormal_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
#abnormalAccuracys.append(coronal_abnormal_valacc[1])

#sagittal_abnormal_out = MRRes_Abnormal_sagittal.fit_generator(sagittal_abnormal, epochs =50,steps_per_epoch= coronalt_length)
sagittal_abnormal_valacc= MRRes_Abnormal_sagittal.evaluate(sagittal_abnormal_val,steps=saggitalv_length ,verbose=1,max_queue_size=50,use_multiprocessing=False)

abnormalAccuracys.append(sagittal_abnormal_valacc[1])

27/26 [==============================] - 11s 423ms/step


In [ ]:
abnormalAccuracys

[0.8333333134651184, 0.8294752836227417, 0.7345678806304932]

In [ ]:
#aclAccuracys=[]
#axial_acl_output = MRRes_Acl.fit_generator(axial_acl, epochs =20,steps_per_epoch= axialt_length)
#axial_acl_valacc = MRRes_Acl.evaluate_generator(axial_acl_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
#aclAccuracys.append(axial_acl_valacc[1]) 

#coronal_acl_out = MRRes_Acl.fit_generator(coronal_acl, epochs =20,steps_per_epoch= coronalt_length)
#coronal_acl_valacc = MRRes_Acl.evaluate_generator(coronal_acl_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
#aclAccuracys.append(coronal_acl_valacc[1])

#sagittal_acl_out = MRRes_Acl.fit_generator(sagittal_acl, epochs =7,steps_per_epoch= saggitalt_length)
sagittal_acl_valacc = MRRes_Acl.evaluate_generator(sagittal_acl_val,steps=saggitalv_length ,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(sagittal_acl_valacc[1])

27/26 [==============================] - 8s 299ms/step


In [ ]:
aclAccuracys

[0.6944444179534912, 0.5617284178733826, 0.5432098507881165]

In [ ]:
#meniscusAccuracys=[]
#axial_meniscus_output = MRRes_Meniscus.fit_generator(axial_acl, epochs =1,steps_per_epoch= axialt_length)
#axial_meniscus_valacc = MRRes_Meniscus.evaluate_generator(axial_acl_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
#meniscusAccuracys.append(axial_acl_valacc[1]) 

coronal_meniscus_out = MRRes_Meniscus.fit_generator(coronal_meniscus, epochs =1,steps_per_epoch= axialt_length)
coronal_meniscus_valacc = MRRes_Meniscus.evaluate_generator(coronal_meniscus_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(coronal_meniscus_valacc[1])

sagittal_meniscus_out = MRRes_Meniscus.fit_generator(sagittal_meniscus, epochs =1,steps_per_epoch= axialt_length)
sagittal_meniscus_valacc = MRRes_Meniscus.evaluate_generator(sagittal_meniscus_val,steps=saggitalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(sagittal_meniscus_valacc[1])

Epoch 1/1
120/120 [==============================] - 15s 129ms/step
Epoch 1/1
1092/1131 [===========================>..] - ETA: 3s - loss: 0.6497 - accuracy: 0.6487

KeyboardInterrupt: ignored

In [ ]:
meniscusAccuracys

[0.5614376068115234, 0.5692083239555359, 0.5648372769355774]

In [ ]:
abnormal_acc = combine_accuracies(abnormalAccuracys)
acl_acc = combine_accuracies(aclAccuracys)
meniscus_acc = combine_accuracies(meniscusAccuracys)